<a id='Top Doc'></a>

# P3: Wrangling OpenStreetMap Data

## Udacity Data Analyst NanoDegree
___
#### Los Angeles, California, USA

### Contents
___

1. [Map Data](#Map-Data)
2. [Data Audit](#Data-Audit)
  1. [Data Structure](#Data-Structure)
  2. [Tag Attributes and Values](#Tag-Attributes-and-Values)
    1. [Node Tags Audit](#Node-tags)
    2. [Way Tags Audit](#Way-tags)
3. [Data Cleaning](#Data-Cleaning)
  1. [Data Check](#Data-Check)
4. [Queries and Overview Statistics](#Queries-and-Overview-Statistics)
  1. [File Sizes](#File-Sizes)
  2. [Number of Documents](#Number-of-Documents)
  3. [General Overview](#General-Overview)
5. [Additional Improvements](#Additional-Improvements)
  1. [Mapping Contributions](#Mapping-Contributions)
  2. [More non-Americanisms](#More-non-Americanisms)
  3. [Automate Drive Throughs?](#Automate-Drive-Throughs?)
6. [Conclusion](#Conclusion)
7. [References](#References)

### Map Data
___

##### Los Angeles, California, USA
[Mapzen](https://mapzen.com/data/metro-extracts/metro/los-angeles_california): `https://mapzen.com/data/metro-extracts/metro/los-angeles_california`

I downloaded the initial los-angeles_california.osm dataset from Mapzen (not included in the repository). The original dataset is 8.6 GB large, and after trying to run some of the code from the courses locally, it crashed my computer. I proceeded to make a series of sample files to use (based off of the sample.py script given in the Project Details), which are described in this table:

##### Sample files used for Project
Name | K-val | Size | Lines | Note
---|---:|---:|---:|---|
*la-sample.osm* | 1,000 | 8.6 MB | 110,729 | This file was used to test early python data auditing and cleaning scripts.
*la-med.osm* | 500 | 17.4 MB | 222,181 | Used as an intermediate test of the data auditing and cleaning scripts. Not included in this repository.
*la-final.osm* | 150 | 58.0 MB | 741,172 | Final file used for auditing, cleaning, and importing into the MongoDB database. Not included in this repository, but can be added for resubmission if necessary.

[Back To Contents](#Contents)

### Data Audit
___
#### Data Structure

After reviewing the [OSM XML Content][1] page, I wanted to check that the sample dataset actually had the tags and structure as described. Mainly that the XML was organized as blocks of ***nodes*** with tags for each node, ***ways*** with tags and references to their respectives nodes, and ***relations*** with tags and references too.

Using and modifying the `mapparser.py` code from the MongoDB Case Study for OSM Data, I generated a dictionary with element tag names, counts, and attributes. A simple table with the results can be seen here:

[1]: #References

Name | Count | Attributes
---|---:|---
member   | 78     | ref, role, type
nd       | 41,121 | ref
tag      | 24,813 | k, v
node     | 37,437 | changeset, id, lat, lon, timestamp, uid, user, version
relation | 32     | changeset, id, timestamp, uid, user, version
way      | 3,655  | changeset, id, timestamp, uid, user, version

I further modified the `mapparser.py` to see the structure of the data. I named it `data_structure.py` and the results are here:
```python
python data_structure.py

{'member': {'count': 78},
 'nd': {'count': 41121},
 'node': {'count': 37437,
          'tag': {'attributes': {'k': 968, 'v': 968}, 'count': 968}},
 'osm': {'count': 1,
         'node': {'attributes': {'changeset': 37437,
                                 'id': 37437,
                                 'lat': 37437,
                                 'lon': 37437,
                                 'timestamp': 37437,
                                 'uid': 37437,
                                 'user': 37437,
                                 'version': 37437},
                  'count': 37437},
         'relation': {'attributes': {'changeset': 32,
                                     'id': 32,
                                     'timestamp': 32,
                                     'uid': 32,
                                     'user': 32,
                                     'version': 32},
                      'count': 32},
         'way': {'attributes': {'changeset': 3655,
                                'id': 3655,
                                'timestamp': 3655,
                                'uid': 3655,
                                'user': 3655,
                                'version': 3655},
                 'count': 3655}},
 'relation': {'count': 32,
              'member': {'attributes': {'ref': 78, 'role': 78, 'type': 78},
                         'count': 78},
              'tag': {'attributes': {'k': 205, 'v': 205}, 'count': 205}},
 'tag': {'count': 24813},
 'way': {'count': 3655,
         'nd': {'attributes': {'ref': 41121}, 'count': 41121},
         'tag': {'attributes': {'k': 23640, 'v': 23640}, 'count': 23640}}}
```

Right away, you can see that ***nodes***, ***ways***, and ***members*** do in fact have the ***tag*** and ***nd*** tags as described by the OSM XML Content page. I also printed out the attributes, and nothing seems out of place. Further, the counts for each of the attributes matches the count of the tags themselves, so I don't have to worry about fixing any of those.

Interestingly, there are some uncertainties mentioned some problems that could merit further investigation if the data cleaning and auditing functions were to be used as a service in any way. Of note are that id or usernames not necessarily being present, untagged unconnected nodes, element IDs that are negative, among others. It would be important to implement a solution to check and correct these problems.

#### Tag Attributes and Values

The attributes for the tags member, nd, node, relation, and way seem to be fairly straightforward and easy to organize, so my Data Cleaning plan will organize those in a simple manner. However, the tag attributes may be a little more difficult to work with. Since the attribute ***k*** represents ***key***, which is assigned by the human user, there can be any number of different values for the k attribute.

Again, I modified an example of the course code to make audit.py, which looks at the k values for tags of the given tag type. I ran it for both ***node*** and ***way*** tags, and found some interesting results. The lists were very long, so I will go over the issues that stand out below. To evaluate specific tag values, I'm using the function `tag_search(filename, tag_name, regex)` in the audit.py file, with an example below. The `regex` value was changed for each tag attribute investigated.

___
##### Node tags
**addr:street** and **addr:street_direction_prefix** - these could be redundant, but I will have to review a few samples to see if this is worth correcting. Running the following code
```python
from audit import tag_search
import pprint

results = tag_search("data/la-small.osm", "node", r'addr:street')
pprint.pprint(results)
```
results in a single tag having the **addr:street_direction_prefix** key. Specifically, `{'k':'addr:street_direction_prefix', 'v': 'W'}`, meaning that I may be able to include this value with it's accompanying **addr:street** tag, if they are tags of the same node. Further, we can use the same code and change the regular expression

**Color** and **Colour** - Simply, the **color** and **colour** keys are going to be the same thing. Since this is LA, I'm going to change all **colour** keys to **color** when cleaning.

**Fixme** - There was one of these tags, and I may just exclude it in cleaning.

In [1]:
from audit import tag_search
import pprint

results = tag_search("data/la-small.osm", "node", r'fixme')
pprint.pprint(results)

[{'k': 'fixme', 'v': 'Transfer_info'}]


**Is_in** - This field returns redundant information, such as the state, country that the nodes are located in. Since I am evaluating Los Angeles, California, all these nodes should be in California, and most definitely in the US. I am going to exclude these tags from the database.

**GNIS** - Finally, it appears that someone has somehow (programmatically or not) included data from the United States Geographic Service - Geographic Names Information Service ([USGS GNIS][1]) as tags for certain nodes. After doing a little research, it appears that in 2009 US GNIS data was bulk imported into OSM. According to the OSM wiki entry, the GNIS is a database of "names" and not "features" and further, that many of these entries are incorrect or no longer exist. This poses a fantastic challenge to OSM, and would be a great opportunity for programmatically cleaning the OSM database, which I will discuss in review below. Using tag_search, it doesn't look like there is much of value in these tags, at least for this project. I am going to exclude these tags as well.

Other than these, every other tag is fairly straightforward.
___
##### Way tags

**FIXME**, **FMMP**, **NHD**, **NHS**, **gnis** - These will be ignored
 
**tiger:** - oddly, I found a series of tags with keys that had **tiger:** in them.
[1]: #References

In [2]:
results = tag_search("data/la-small.osm", "way", r'tiger:')
pprint.pprint(results[:6])

[{'k': 'tiger:cfcc', 'v': 'A74'},
 {'k': 'tiger:tlid', 'v': '195710849:195710852'},
 {'k': 'tiger:county', 'v': 'San Diego, CA'},
 {'k': 'tiger:source', 'v': 'tiger_import_dch_v0.6_20070809'},
 {'k': 'tiger:reviewed', 'v': 'no'},
 {'k': 'tiger:upload_uuid',
  'v': 'bulk_upload.pl-5dac241b-d144-4c9c-9e26-b4dec4590a61'}]


these look like some sort of algorithmic utility for uploading data to OSM. Further investigation proves this to be correct, as TIGER stands for "The Topologically Integrated Geographic Encoding and Referencing system (TIGER) data,[which is] produced by the US Census Bureau, is a public domain data source which has many geographic features. The TIGER/Line files are extracts of selected geographic information, including roads, boundaries, and hydrography features. All of the roads were imported into OSM in 2007 and 2008, populating the nearly empty map of the United States." from the [OSM wiki page][1]. I am going to exclude these as well, as the wiki explains that much of the US mapping is now done by the OSM mapping community, as mass uploads of the TIGER data stopped after 2007, and should be unimportant to the analysis for this project.

Other than these, the other tags should clean well, and any issues in the tags' key values will have to be the subject of a secondary cleaning.

[1]: #References

### Data Cleaning
___

I modified the data.py code from the final quiz in the MongoDB for OSM Case study as a base script for shaping and converting the osm data into json. After running the cleaning script on la-small.osm, I used the command line to do a basic check of the resulting json file.

```shell
python data_converter.py | less
```

which results in the following:

```
[{'created': {'changeset': '12626769',
              'timestamp': '2012-08-05T21:43:42Z',
              'uid': '104962',
              'user': 'techlady',
              'version': '10'},
  'id': '653656',
  'pos': [33.9598406, -118.3770539],
  'type': 'node'},
 {'created': {'changeset': '397625',
              'timestamp': '2009-02-13T13:10:51Z',
              'uid': '96974',
              'user': 'mattmaxon',
              'version': '75'},
  'id': '15711929',
  'pos': [34.4277824, -118.4172198],
  'type': 'node'},...
```

so everything looks ok so far. I imported the json into mongoDB to run some basic tests of the resulting database. The import resulted in 41,124 documents:
```shell
~/Dropbox/DAND/P3/data (master)(dandy) λ mongoimport -d osm -c la --file la-small.osm.json
2017-03-09T15:48:12.410-0800    connected to: localhost
2017-03-09T15:48:13.875-0800    imported 41124 documents
```

which I double checked by running a query in the mongo command line environment:

```shell
> use osm
> db.la.find().count()
41124
```

Which confirmed the document count. Also, this matched the total **node**, **way**, and **relation** tags in the [Data Structure table][1] above. I thought it was interesting that the OSM database in mongoDB was 4 MB large, whereas the imported la-small.osm.json file was 9.5 MB. Clearly MongoDB is doing something under the hood to make storage a lot more efficient.

#### Data Check
I wanted to confirm that the osm data was cleaned and mapped to json properly, so I ran a few queries to find a couple sample **node**, **way**, and **relation** nodes. I

```shell
> db.la.find({"type":"node"})
> db.la.find({"type":"way"})
> db.la.find({"type":"relation"})
```

the **node** and **way** queries found some entries, and on a simple pass through, looked fine. However, there was a problem with the **relation** tags. None came up. So I created `query.py` (bsed on the OSM Case Study) and defined the `agg_types` function to print out "type" of documents stored in the database:

```shell
> python query.py agg_types sort
[{u'_id': u'node', u'count': 37437},
 {u'_id': u'way', u'count': 3654},
 {u'_id': u'multipolygon', u'count': 25},
 {u'_id': u'restriction', u'count': 8}]
```

which resulted in an interesting problem. It turns out that some **tag** nodes have a **k** called "type" which was overwriting the "type" of the **node**, **way**, and **relation** nodes. I fixed this by changing the name of the node "types" to "tag_type" in the data_converter.py script. After re-creating the la-small json file, and reimporting it to mongoDB, I reran the query.py script:
```shell
> python query.py agg_types sort
[{u'_id': u'node', u'count': 37437},
 {u'_id': u'way', u'count': 3655},
 {u'_id': u'relation', u'count': 32}]
```

and the results are now as expected. I ran a quick check of the "color" tags in the mongo environment.:
```shell
> db.la.find({"color":{"$exists":1}})
```
it turns out that my data_converter.py script was not successfully cleaning the "colour" key. I made a change then re-ran the script.

```shell
{ "_id" : ObjectId("58c2037bdb84613cc7fb4281"), "material" : "metal", "created" : { "uid" : "2005182", "changeset" : "22157076", "version" : "1", "user" : "frankthetankk", "timestamp" : "2014-05-05T23:35:37Z" }, "color" : "red", "backrest" : "no", "pos" : [ 33.9180429, -117.5693455 ], "leisure" : "picnic_table", "seats" : "12", "covered" : "no", "id" : "2838745246", "tag_type" : "node" }
{ "_id" : ObjectId("58c2037bdb84613cc7fb4300"), "emergency" : "fire_hydrant", "created" : { "uid" : "416346", "changeset" : "24452864", "version" : "1", "user" : "Brian@Brea", "timestamp" : "2014-07-30T21:55:16Z" }, "color" : "yellow", "pos" : [ 33.8738925, -117.92132 ], "id" : "2988542221", "tag_type" : "node" }
```

Perfect.

Finally, I checked for any of the tags I wanted to skip, mainly all "fixme", "FIXME", "is_in", "gnis", "FMMP", "NHD", "NHS", and "tiger".

```script
> db.la.find({"fixme":{"$exists":1}})
> db.la.find({"FIXME":{"$exists":1}})
> db.la.find({"is_in":{"$exists":1}})
> db.la.find({"gnis":{"$exists":1}})
> db.la.find({"FMMP":{"$exists":1}})
> db.la.find({"NHD":{"$exists":1}})
> db.la.find({"NHS":{"$exists":1}})
> db.la.find({"tiger":{"$exists":1}})
```

None of the queries returned any results. Success!


[1]: #Data-Structure

[Back To Contents](#Contents)

### Queries and Overview Statistics
___

After all the above, I dropped the database, and re-ran my python scripts with the la-final.osm dataset, which I am going to use for my data investigation. There were no errors, and the numbers for count, etc. were simply larger than the sample dataset numbers.

#### File Sizes
File Name | Size | Lines | Notes
---|---|---|---
la-final.osm | 58.0 MB | 741,172 | 
la-final.osm.json | 64.7 MB | 274,157 | Amazingly, converting to json shrinks the lines by roughly 63%, but increase the file size

##### Number of Documents
Also, amazingly, the mongoDB osm database is roughly 23 MB, which is a lot smaller the original datasets themselves

```shell
> db.la.find().count()
274,157
```

There are 274,157 documents in this database

##### General Overview
Query for unique users:
```
> db.la.distinct("created.user").length
1234
```

Number of nodes, ways, and relations:
```
> db.la.aggregate([{"$group" : {"_id" : "$tag_type", "count" : {"$sum" : 1 }}}, {"$sort" : {"count" : -1}}])
[{u'_id': u'node', u'count': 249577},
 {u'_id': u'way', u'count': 24365},
 {u'_id': u'relation', u'count': 215}]
```

Top contributor (makes up roughly 7.0% of all contribtions):
```
> db.la.aggregate([{"$group" : {"_id" : "$created.user", "count" : {"$sum" : 1 }}}, {"$sort" : {"count" : -1}}, {"$limit" : 1}])
[{u'_id': u'manings_labuildings', u'count': 19127}]
```

Top 5 shop types in data:
```
> db.la.aggregate([{"$group" : {"_id" : "$shop", "count" : {"$sum":1}}}, {"$sort" : {"count" : -1}},
> {"$limit" : 6}])
{ "_id" : null, "count" : 273885 }
{ "_id" : "department_store", "count" : 6 }
{ "_id" : "supermarket", "count" : 5 }
{ "_id" : "convenience", "count" : 5 }
{ "_id" : "clothes", "count" : 4 }
{ "_id" : "mall", "count" : 3 }
```

Top 5 amenities in data:
```
> db.la.aggregate([{"$group" : {"_id" : "$amenity", "count" : {"$sum":1}}}, {"$sort" : {"count" : -1}}, {"$limit" : 6}])
{ "_id" : null, "count" : 273885 }
{ "_id" : "parking", "count" : 79 }
{ "_id" : "place_of_worship", "count" : 31 }
{ "_id" : "school", "count" : 26 }
{ "_id" : "restaurant", "count" : 25 }
{ "_id" : "fast_food", "count" : 11 }
```
Hahaha, parking. Soo LA. Also along that vein...

Top 10 speed limits:
```
> db.la.aggregate([{"$group" : {"_id" : "$maxspeed", "count" : {"$sum":1}}}, {"$sort" : {"count" : -1}}, {"$limit" : 11}])
{ "_id" : null, "count" : 273854 }
{ "_id" : "35 mph", "count" : 71 }
{ "_id" : "40 mph", "count" : 54 }
{ "_id" : "45 mph", "count" : 49 }
{ "_id" : "50 mph", "count" : 30 }
{ "_id" : "65 mph", "count" : 23 }
{ "_id" : "25 mph", "count" : 21 }
{ "_id" : "55 mph", "count" : 16 }
{ "_id" : "30 mph", "count" : 11 }
{ "_id" : "15 mph", "count" : 6 }
{ "_id" : "14 mph", "count" : 3 }
```

Overview of Landuse:
```
> db.la.aggregate([{"$group" : {"_id" : "$landuse", "count" : {"$sum":1}}}, {"$sort" : {"count" : -1}}, {"$limit" : 11}])
{ "_id" : null, "count" : 274009 }
{ "_id" : "residential", "count" : 46 }
{ "_id" : "grass", "count" : 35 }
{ "_id" : "meadow", "count" : 15 }
{ "_id" : "industrial", "count" : 11 }
{ "_id" : "retail", "count" : 7 }
{ "_id" : "forest", "count" : 6 }
{ "_id" : "farmyard", "count" : 6 }
{ "_id" : "commercial", "count" : 5 }
{ "_id" : "farm", "count" : 3 }
{ "_id" : "orchard", "count" : 2 }
```

[Back To Contents](#Contents)

### Additional Improvements
___


#### Mapping Contributions
An interesting visualization would be a "heat-map" of contributions. Since nodes and ways have latitude and longitude attributes, we could generate a list of lat-lon pairs, and maps those as dots on a blank canvas. You could even run a query to count any similar coordinates, and adjust the size of the dots accordingly. I think this would be a fascinating way to see where most of the user activity is for the OSM file. Further, it would be interesting to see if the points themselves can generate a dot-map of the city. I don't know how to run the visualization yet, but I can build a query which pulls coordinates and counts

```
> db.la.aggregate([{"$group" : {"_id" : "$pos", "count" : {"$sum" : 1}}}])
{ "_id" : null, "count" : 24580 }
{ "_id" : [ 33.7477613, -117.1908992 ], "count" : 1 }
{ "_id" : [ 34.2833768, -118.5340969 ], "count" : 1 }
{ "_id" : [ 34.2814896, -118.5348974 ], "count" : 1 }
Type "it" for more
```

#### More non-Americanisms
So, while doing some exploratory queries, I stumbled upon this:
```
> db.la.aggregate([{"$group" : {"_id" : "$leisure", "count" : {"$sum":1}}}, {"$sort" : {"count" : -1}}, {"$limit" : 11}])
{ "_id" : null, "count" : 273985 }
{ "_id" : "pitch", "count" : 45 }
{ "_id" : "swimming_pool", "count" : 38 }
{ "_id" : "garden", "count" : 35 }
```
***PITCH?!*** is super British. Minus 10 points for Gryffindor. In America, we simply call them fields. I'm all for globalization and learning about new things, but if this data is to accurately reflect America, then the map lexicon should reflect that too. It's kind of like when you do a global view of [Google Maps][1], where international cities also include the name in the language as well (Asia chosen as an example, because of the diversity of language). Going along with this vein, it might be interesting to set up a sort of lexicon filter, to automate the nationalization, if you will, of the maps being cleaned. The difficulty in doing this is that the lexicon might be tremendously subject to the user's opinion on the matter. As an example, I think all pitches in America should be called fields, but that is certainly up for debate.

#### Automate Drive Throughs?
I also discovered this beauty:
```
> db.la.aggregate([{"$group" : {"_id" : "$drive_through", "count" : {"$sum":1}}}, {"$sort" : {"count" : -1}}, {"$limit" : 11}])
{ "_id" : null, "count" : 274154 }
{ "_id" : "yes", "count" : 2 }
{ "_id" : "no", "count" : 1 }
```
I would love to find away to automate the updating of these tags for all stores and amenities.


[1]: https://www.google.com/maps/@20.4331742,97.1758595,6z

[Back To Contents](#Contents)

### Conclusion
___

This has been a really fun project. Diving through the data for LA really shows you the character of the city, especially because the customizability of OSM reflects the values of its users. Do you want to track horses on a property? You can do so. Do you want to know if a store is a drive through? By all means. There is so much to explore in this data, and scripting solutions for cleaning and auditing the data has been a blast.

Thanks for your consideration!

[Back To Contents](#Contents)

### References
___

1. [OSM: XML Contents wiki](https://wiki.openstreetmap.org/wiki/OSM_XML#Contents): `https://wiki.openstreetmap.org/wiki/OSM_XML#Contents`
2. [OSM: USGS GSM wiki](http://wiki.openstreetmap.org/wiki/USGS_GNIS): `http://wiki.openstreetmap.org/wiki/USGS_GNIS`
3. [OSM: TIGER wiki](http://wiki.openstreetmap.org/wiki/TIGER): `http://wiki.openstreetmap.org/wiki/TIGER`
___
[Back To Contents](#Contents)